<a href="https://colab.research.google.com/github/lani009/IDS-DataMining/blob/main/%5BDM%5DApriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
data = pd.read_csv('DM_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  int64  
 2   service                      25192 non-null  int64  
 3   flag                         25192 non-null  int64  
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   hot                          25192 non-null  int64  
 9   num_failed_logins            25192 non-null  int64  
 10  logged_in                    25192 non-null  int64  
 11  num_compromised              25192 non-null  int64  
 12  root_shell                   25192 non-null  int64  
 13  su_attempted    

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
from mlxtend.frequent_patterns import apriori,association_rules

In [5]:
sc = StandardScaler()
sc_data = sc.fit_transform(data)

sc_df = pd.DataFrame(sc_data, columns=data.columns)
sc_df.head(n=10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,index_num
0,-0.113551,-0.444009,-1.399448,0.744553,-0.009889,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.720244,-0.354628,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.349282,-0.373886,-0.328634,-0.813985,-0.779157,-0.280673,0.073120,-0.287993,-0.641804,-0.627365,-0.221668,-0.374281,-0.934425,-1.731982
1,-0.113551,1.325565,0.780883,0.744553,-0.010032,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.624317,-0.368427,-0.640142,-0.633978,-0.372186,-0.373098,-1.320567,0.490836,-0.373886,0.732059,-1.030895,-1.157831,2.764403,2.375620,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281,-0.934425,-1.731845
2,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.334947,-0.299430,1.595477,1.600209,-0.372186,-0.373098,-1.388806,0.042773,-0.373886,0.732059,-0.804947,-0.935081,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731707
3,-0.113551,-0.444009,0.780883,0.744553,-0.009996,0.052473,-0.00891,-0.091223,-0.091933,-0.02622,1.238197,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.694082,-0.313230,-0.193018,-0.187141,-0.372186,-0.373098,0.772109,-0.349282,-0.373886,-1.540854,1.264742,1.069663,-0.440940,-0.380894,0.073759,-0.574435,-0.604947,-0.385140,-0.342768,-0.934425,-1.731570
4,-0.113551,-0.444009,0.780883,0.744553,-0.010010,-0.034582,-0.00891,-0.091223,-0.091933,-0.02622,1.238197,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.476067,0.059355,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.349282,-0.023115,0.732059,1.264742,1.069663,-0.440940,-0.478183,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281,-0.934425,-1.731432
5,-0.113551,-0.444009,-1.377199,-2.025203,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.317506,-0.120038,-0.640142,-0.633978,2.765176,2.729322,-1.138595,-0.013235,-0.373886,0.732059,-0.868212,-1.001906,-0.066984,-0.478183,-0.287993,-0.641804,-0.627365,2.884296,2.777041,1.070177,-1.731295
6,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.709933,-0.258032,1.595477,1.600209,-0.372186,-0.373098,-1.388806,-0.013235,-0.373886,0.732059,-0.958592,-1.068731,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731157
7,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.282624,-0.161436,1.595477,1.600209,-0.372186,-0.373098,-1.184088,-0.013235,-0.373886,0.732059,-0.904364,-1.024181,-0.066984,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731019
8,-0.113551,-0.444009,0.780883,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,1.616874,-0.064840,1.595477,1.600209,-0.372186,-0.373098,-1.297820,-0.069243,-0.373886,0.732

In [6]:
def encode_units(x):
  if x <= 0 :
    return 0
  if x >= 0 :
    return 1

train_df = sc_df.applymap(encode_units)

train_df.head(n=10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,index_num
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0
3,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0


In [7]:
data_X = train_df.drop(columns = ["index_num"])

X_train, X_test = train_test_split(data_X, test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape)

(16878, 39) (8314, 39)


In [8]:
df = pd.DataFrame(X_train, columns=data.drop(columns = ["index_num"]).columns)

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
14666,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1
10743,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0
2487,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1
21251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1
7387,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0


In [9]:
frequent_itemsets = apriori( df, min_support = 0.01, use_colnames=True, max_len =2)
result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc

,support,itemsets
18,0.642612,(dst_host_count)
15,0.621519,(same_srv_rate)
2,0.619327,(service)
3,0.610973,(flag)
90,0.566833,"(flag, same_srv_rate)"
...,...,...
112,0.010724,"(dst_bytes, dst_host_same_src_port_rate)"
129,0.010606,"(num_compromised, same_srv_rate)"
38,0.010487,"(dst_host_rerror_rate, duration)"
85,0.010191,"(num_compromised, flag)"


In [10]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.90)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
61,(num_compromised),(logged_in),0.010724,0.392464,0.010724,1.000000,2.548007,0.006515,inf
31,(protocol_type),(flag),0.186515,0.610973,0.186515,1.000000,1.636734,0.072559,inf
60,(num_compromised),(service),0.010724,0.619327,0.010724,1.000000,1.614656,0.004082,inf
37,(srv_count),(flag),0.129814,0.610973,0.129281,0.995892,1.630011,0.049968,94.706495
11,(srv_serror_rate),(serror_rate),0.285164,0.287771,0.283920,0.995637,3.459823,0.201858,163.236089
...,...,...,...,...,...,...,...,...,...
9,(logged_in),(service),0.392464,0.619327,0.359403,0.915761,1.478639,0.116340,4.518958
53,(dst_bytes),(dst_host_same_srv_rate),0.097583,0.498163,0.089288,0.914997,1.836741,0.040676,5.903750
59,(src_bytes),(flag),0.012798,0.610973,0.011672,0.912037,1.492762,0.003853,4.422624
1,(same_srv_rate),(flag),0.621519,0.610973,0.566833,0.912011,1.492720,0.187101,4.421338


In [11]:
rules[rules['consequents'] == {"class"}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
26,(dst_host_srv_serror_rate),(class),0.281017,0.467295,0.276277,0.983133,2.103882,0.144959,31.582765
20,(srv_serror_rate),(class),0.285164,0.467295,0.277047,0.971535,2.079064,0.143791,18.714676
17,(serror_rate),(class),0.287771,0.467295,0.277580,0.964587,2.064195,0.143106,15.042730
25,(dst_host_serror_rate),(class),0.287060,0.467295,0.276514,0.963261,2.061357,0.142372,14.499759


In [12]:
test = pd.DataFrame(X_test, columns=data.drop(columns = ["index_num"]).columns)
test.head(n=10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
19064,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1
11127,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,1,0
6517,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1
2973,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
13339,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1
19289,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0
2166,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
5548,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
10887,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0
2222,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1


In [13]:
col = ['dst_host_srv_serror_rate', 'srv_serror_rate', 'serror_rate', 'dst_host_serror_rate']

In [14]:
idx_1 = test[test['dst_host_srv_serror_rate'] == 0 ].index
test_df = test.drop(idx_1)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)


(2348, 39)
(2317, 39)


In [15]:
idx_2 = test[test['srv_serror_rate'] == 0 ].index
test_df = test.drop(idx_2)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)

(2403, 39)
(2320, 39)


In [16]:
idx_3 = test[test['serror_rate'] == 0 ].index
test_df = test.drop(idx_3)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)

(2412, 39)
(2323, 39)


In [17]:
idx_4 = test[test['dst_host_serror_rate'] == 0 ].index
test_df = test.drop(idx_4)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)

(2398, 39)
(2316, 39)


In [18]:
idx_a = test[(test['dst_host_srv_serror_rate'] == 0) & (test['srv_serror_rate'] == 0) & (test['serror_rate'] == 0) & (test['dst_host_serror_rate'] == 0)].index
test_df = test.drop(idx_a)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)


(2499, 39)
(2338, 39)


In [19]:
idx_b = test[(test['dst_host_srv_serror_rate'] == 0) | (test['srv_serror_rate'] == 0) | (test['serror_rate'] == 0) | (test['dst_host_serror_rate'] == 0)].index
test_df = test.drop(idx_b)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)

(2301, 39)
(2300, 39)


*Apriori test accuracy = 97.03%*